# --- Day 7: Some Assembly Required ---
This year, Santa brought little Bobby Tables a set of wires and bitwise logic gates! Unfortunately, little Bobby is a little under the recommended age range, and he needs help assembling the circuit.

Each wire has an identifier (some lowercase letters) and can carry a 16-bit signal (a number from 0 to 65535). A signal is provided to each wire by a gate, another wire, or some specific value. Each wire can only get a signal from one source, but can provide its signal to multiple destinations. A gate provides no signal until all of its inputs have a signal.

The included instructions booklet describes how to connect the parts together: x AND y -> z means to connect wires x and y to an AND gate, and then connect its output to wire z.

For example:

123 -> x means that the signal 123 is provided to wire x.
x AND y -> z means that the bitwise AND of wire x and wire y is provided to wire z.
p LSHIFT 2 -> q means that the value from wire p is left-shifted by 2 and then provided to wire q.
NOT e -> f means that the bitwise complement of the value from wire e is provided to wire f.
Other possible gates include OR (bitwise OR) and RSHIFT (right-shift). If, for some reason, you'd like to emulate the circuit instead, almost all programming languages (for example, C, JavaScript, or Python) provide operators for these gates.

For example, here is a simple circuit:

123 -> x;
456 -> y;
x AND y -> d;
x OR y -> e;
x LSHIFT 2 -> f;
y RSHIFT 2 -> g;
NOT x -> h;
NOT y -> i;

After it is run, these are the signals on the wires:

d: 72;
e: 507;
f: 492;
g: 114;
h: 65412;
i: 65079;
x: 123;
y: 456;

In little Bobby's kit's instructions booklet (provided as your puzzle input), what signal is ultimately provided to wire a?

# Note: Part 2

Exactly the same using a different input file

In [35]:
# Read input

import pandas as pd
import logging

df = pd.read_csv(r"C:\Users\Jono\Documents\Python Scripts\Day 7 Input P2.csv")

steps = []

for row in df["step"]:
    steps += [row] # .append cannot be used with strings, so must use += []

In [36]:
# Define a dictionary of values

signals = {}

def run(steps, signals, count):
    count = 0
    for step in steps:
        split = str.split(step)
        if 'NOT' in split:
            signals, count = NOTGate(step, signals, count)
        elif 'OR' in split:
            signals, count = ORGate(step, signals, count)
        elif 'AND' in split:
            signals, count = ANDGate(step, signals, count)
        elif 'RSHIFT' in split:
            signals, count = RSGate(step, signals, count)
        elif 'LSHIFT' in split:
            signals, count = LSGate(step, signals, count)
        elif '->' in split:
            signals, count = noGate(step, signals, count)
        else:
            raise Exception('Unrecognised input.')
    
    return signals, count

In [37]:
# Define functions for each operator, which read the shape of the step, 
    # define values that are not already in the dictionary, then set the values
    
#No Gate - 
    # Shape nnnnn -> ll (up to 5 digit number, up to 2 letter alpha)

import logging
    
logging.disable(logging.DEBUG)
    
def noGate(step, dic, exc):
    list = str.split(step) # splits into a list containing [nnnnn, ->, ll]
    try:
        list[0] = int(list[0])
    except:
        pass
    
    if list[2] not in dic:
        if isinstance(list[0], int) == True:
            dic[list[2]] = list[0]
            logging.debug(step)
        elif list[0] in dic:
            dic[list[2]] = dic[list[0]]
            logging.debug(step)
        else:
            exc = exc + 1
        
    return dic, exc        
    
#NOT Gate - CHECKED OK
    # Shape NOT ll -> ll

def NOTGate(step, dic, exc):
    list = str.split(step) # splits into a list containing [NOT, ll, ->, ll]
    try:
        list[1] = int(list[1])
    except:
        pass

    if list[3] not in dic:
        if isinstance(list[1], int) == True:
            newSignal = ~(list[1])
            dic[list[3]] = newSignal
            logging.debug(step)            
        elif list[1] in dic:
            newSignal = ~(dic[list[1]]) 
            dic[list[3]] = newSignal
            logging.debug(step)
        else:
            exc = exc + 1

    return dic, exc

#OR Gate - CHECKED OK
    # Shape ll OR ll -> ll
    
def ORGate(step, dic, exc):
    list = str.split(step) # splits into a list containing [ll, OR, ll, ->, ll]
    try:
        list[0] = int(list[0])
    except:
        pass
        
    try:
        list[2] = int(list[2])
    except:
        pass
        
    if list[4] not in dic:
        if isinstance(list[0], int) and isinstance(list[2], int) == True:
            newSignal = (list[0] | list[2]) 
            dic[list[4]] = newSignal
            logging.debug(step)
        elif isinstance(list[0], int) == True and list[2] in dic:
            newSignal = (list[0] | dic[list[2]]) 
            dic[list[4]] = newSignal
            logging.debug(step)
        elif isinstance(list[2], int) == True and list[0] in dic:
            newSignal = (dic[list[0]] | list[2]) 
            dic[list[4]] = newSignal
            logging.debug(step)
        elif list[0] in dic:
            if isinstance(list[2], int) == True:
                newSignal = (dic[list[0]] | list[2]) 
                dic[list[4]] = newSignal
                logging.debug(step)
            elif list[2] in dic:      
                print(list[0], list[2])
                newSignal = (dic[list[0]] | dic[list[2]]) 
                dic[list[4]] = newSignal
                logging.debug(step)
        else:
            exc = exc + 1
        
    return dic, exc
        
#AND Gate - CHECKED OK
    # Shape ll AND ll -> ll
    
def ANDGate(step, dic, exc):
    list = str.split(step) # splits into a list containing [ll, AND, ll, ->, ll]
    try:
        list[0] = int(list[0])
    except:
        pass
        
    try:
        list[2] = int(list[2])
    except:
        pass
    
    if list[4] not in dic:
        if isinstance(list[0], int) and isinstance(list[2], int) == True:
            newSignal = (list[0] & list[2]) 
            dic[list[4]] = newSignal
            logging.debug(step)
        elif isinstance(list[0], int) == True and list[2] in dic:
            newSignal = (list[0] & dic[list[2]]) 
            dic[list[4]] = newSignal
            logging.debug(step)
        elif list[0] in dic:
            if isinstance(list[2], int) == True:
                newSignal = (dic[list[0]] & list[2]) 
                dic[list[4]] = newSignal
                logging.debug(step)
            elif list[2] in dic:      
                print(list[0], list[2])
                newSignal = (dic[list[0]] & dic[list[2]]) 
                dic[list[4]] = newSignal
                logging.debug(step)
        else:
            exc = exc + 1
        
    return dic, exc

#RSHIFT Gate - CHECKED OK
    # Shape ll RSHIFT nn -> ll
    
def RSGate(step, dic, exc):
    list = str.split(step) # splits into a list containing [ll, RSHIFT, nn, ->, ll]
    
    if list[4] not in dic:
        if list[0] in dic:
            newSignal = (dic[list[0]] >> int(list[2])) 
            dic[list[4]] = newSignal
            logging.debug(step)
        else:
            exc = exc + 1
        
    return dic, exc

#LSHIFT Gate - CHECKED OK
    # Shape ll LSHIFT nn -> ll
    
def LSGate(step, dic, exc):
    list = str.split(step) # splits into a list containing [ll, LSHIFT, nn, ->, ll]
    
    if list[4] not in dic:
        if list[0] in dic:
            newSignal = (dic[list[0]] << int(list[2])) 
            dic[list[4]] = newSignal
            logging.debug(step)
        else:
            exc = exc + 1
        
    return dic, exc

In [38]:
counter = 1

while counter != 0:
    signals, counter = run(steps, signals, counter)

print(signals)

e f
e f
g i
d j
d j
k m
b n
b n
o q
t s
v w
z aa
z aa
ab ad
y ae
y ae
af ah
x ai
x ai
aj al
ao an
aq ar
au av
au av
aw ay
at az
at az
ba bc
as bd
as bd
be bg
bj bi
bl bm
bp bq
bp bq
br bt
bo bu
bo bu
bv bx
bn by
bn by
bz cb
ce cd
cg ch
ck cl
ck cl
cm co
cj cp
cj cp
cq cs
ci ct
ci ct
cu cw
cz cy
db dc
df dg
df dg
dh dj
de dk
de dk
dl dn
dd do
dd do
dp dr
du dt
dw dx
ea eb
ea eb
ec ee
dz ef
dz ef
eg ei
dy ej
dy ej
ek em
ep eo
er es
ev ew
ev ew
ex ez
eu fa
eu fa
fb fd
et fe
et fe
ff fh
fk fj
fm fn
fq fr
fq fr
fs fu
fp fv
fp fv
fw fy
fo fz
fo fz
ga gc
gh gi
gf ge
gl gm
gl gm
gn gp
gk gq
gk gq
gr gt
gj gu
gj gu
gv gx
ha gz
hc hd
hg hh
hg hh
hi hk
hf hl
hf hl
hm ho
he hp
he hp
hq hs
hv hu
hx hy
ib ic
ib ic
id if
ia ig
ia ig
ih ij
hz ik
hz ik
il in
iq ip
is it
iw ix
iw ix
iy ja
iv jb
iv jb
jc je
iu jf
iu jf
jg ji
jn jo
jl jk
jr js
jr js
jt jv
jq jw
jq jw
jx jz
jp ka
jp ka
kb kd
kg kf
ki kj
km kn
km kn
ko kq
kl kr
kl kr
ks ku
kk kv
kk kv
kw ky
lb la
ld le
lh li
lh li
lj ll
lg lm
lg lm
ln lp
lf

# Testing of functions

In [164]:
# Test function noGate

step = "400 -> a"

print(signals)
print(counter)

signals, counter = noGate(step, signals, counter)

print(signals)
print(counter)

{}
2
{'a': 400}
2


In [166]:
# Test function noGate

step = "a -> b"

print(signals)
print(counter)

signals, counter = noGate(step, signals, counter)

print(signals)
print(counter)

{'a': 400, 'b': 400}
2
Input is another variable
{'a': 400, 'b': 400}
2


In [157]:
# Test function NOTGate

step = 'NOT b -> d'

print(signals)
print(counter)

signals, counter = NOTGate(step, signals, counter)

print(signals)
print(counter)

{'b': 400}
1
{'b': 400, 'd': -401}
1


In [78]:
# Test function ORGate

step = 'b OR d -> e'

print(signals)
print(counter)

signals, counter = ORGate(step, signals, counter)

print(signals)
print(counter)

{'b': 44430, 'd': -44431, 'e': -1}
2
{'b': 44430, 'd': -44431, 'e': -1}
2


In [92]:
# Test function ANDGate

step = 'b AND d -> e'

print(signals)
print(counter)

signals, counter = ANDGate(step, signals, counter)

print(signals)
print(counter)

{'b': 400, 'd': -401}
2
{'b': 400, 'd': -401, 'e': 0}
2


In [111]:
# Test function RSGate

step = 'b RSHIFT 2 -> e'

print(signals)
print(counter)

signals, counter = RSGate(step, signals, counter)

print(signals)
print(counter)

{'b': 400}
1
{'b': 400, 'e': 100}
1


In [228]:
# Test function LSGate

step = 'b LSHIFT 2 -> e'

print(signals)
print(counter)

signals, counter = LSGate(step, signals, counter)

print(signals)
print(counter)

{'b': 44430, 'f': 1388, 'e': 5553, 'c': 0, 'g': 5629, 't': 0, 'd': 11107, 'v': 22215, 'h': 1312, 'i': -1313, 'j': 4317, 'k': 15359, 'l': 65, 'm': -66, 'n': 15294, 'o': 49086, 'p': 10638, 'q': -10639, 'r': 38448, 's': 0, 'w': 0, 'u': 0, 'ao': 0, 'x': 22215, 'y': 5553, 'aa': 694, 'aq': 11107}
1
{'b': 44430, 'f': 1388, 'e': 5553, 'c': 0, 'g': 5629, 't': 0, 'd': 11107, 'v': 22215, 'h': 1312, 'i': -1313, 'j': 4317, 'k': 15359, 'l': 65, 'm': -66, 'n': 15294, 'o': 49086, 'p': 10638, 'q': -10639, 'r': 38448, 's': 0, 'w': 0, 'u': 0, 'ao': 0, 'x': 22215, 'y': 5553, 'aa': 694, 'aq': 11107}
1


# Buggy OR function

In [ ]:
def ORGate(step, dic, exc):
    list = str.split(step) # splits into a list containing [ll, OR, ll, ->, ll]
    try:
        list[0] = int(list[0])
    except:
        pass
        
    try:
        list[2] = int(list[2])
    except:
        pass
        
    if list[4] not in dic:
        if isinstance(list[0], int) and isinstance(list[2], int) == True:
            newSignal = (list[0] | list[2]) 
            dic[list[4]] = newSignal
            print(step)
        elif isinstance(list[0], int) == True and list[2] in dic:
            newSignal = (list[0] | dic[list[2]]) 
            dic[list[4]] = newSignal
            print(step)
        elif isinstance(list[2], int) == True and list[0] in dic:
            newSignal = (dic[list[0]] | list[2]) 
            dic[list[4]] = newSignal
            print(step)
        elif list[0] 
            print(list[0], list[2])
            newSignal = (dic[list[0]] | dic[list[2]]) 
            dic[list[4]] = newSignal
            print(step)
        else:
            exc = exc + 1
        
    return dic, exc